[https://www.acmicpc.net/problem/5399](https://www.acmicpc.net/problem/5399)

## Problem

Here you go — a **full English translation** of the problem, rewritten cleanly and formatted in **Markdown**, based entirely on the fetched page content [acmicpc.net](https://www.acmicpc.net/problem/5399).

---

# **🐧 Penguin Bashing — Problem 5399 (English, Markdown Version)**

## **Problem Description**

In a well‑known online game, you play as a yeti whose goal is to hit a cute and cuddly penguin so that it flies as far as possible.

The game takes place in sunny Antarctica, where the ground is littered with obstacles left behind by human and robotic expeditions.

### **Obstacles**

- **Spikes**  
  Sharp, pointy metal objects.  
  Extremely dangerous — if the penguin hits one, the game ends immediately with a score of **–100**.

- **Mines**  
  Left over from various wars in Antarctica.  
  Bad for the penguin’s health, but good for distance:
  - The penguin is flung **2 m forward** through the air.
  - The penguin’s **velocity instantly increases by 4 m/s forward**.

### **Physics**

- The yeti hits the penguin at an **integral angle** between **–90° and +90°**
  - –90° = straight down
  - 0° = horizontal to the right
  - +90° = straight up
- The penguin always receives a **total velocity of 25 m/s**, regardless of angle.
- The penguin is struck at **1 meter above the ground**.
- After landing, the penguin **slides** along the ground with friction:
  - It loses **5 m/s of speed per meter** traveled.
- Gravity is **9.81 m/s²**.
- Air drag is ignored.

### **Scoring**

- If the penguin hits a **spike** → score = **–100**.
- Otherwise → score = **horizontal distance traveled**, in **centimeters**, rounded to the nearest integer.

---

## **Input**

The first line contains a positive integer:  
**T** — the number of test cases.

For each test case:

1. A line with an integer

   $$
   n < 10^5
   $$

   the number of obstacles.

2. Then **n lines**, each containing:
   - A floating‑point number \(x_i\):  
     the position of the obstacle in **meters** from the start.
   - A string: `"mine"` or `"spike"`.

---

## **Output**

For each test case, output **one line** containing:

```
<best_score> <best_angle>
```

- **best_score** — the highest possible score
- **best_angle** — the integer angle (in degrees) that achieves that score
- If multiple angles give the same score, output the **lowest** angle.

---

## **Example Input**

```
1
3
0.1 spike
2 mine
64.7 spike
```

## **Example Output**

```
6829 44
6466 34
7022 39
```


Absolutely — here is your **clean physics model** turned into a **proper Markdown page**, formatted as if it were a standalone document. You can drop this directly into GitHub, Obsidian, or any Markdown viewer.

---

# **📘 Penguin Physics Model — Clean Markdown Page**

## **1. Initial Conditions**

- **Launch speed**
- $$
  v_0 = 25
  $$

- **Launch angle**
  $$

\theta \in [-90^\circ, 90^\circ]

$$

- **Initial height**
$$

y_0 = 1

$$

- **Gravity**
$$

g = 9.81

$$

- **Assumptions**
- No air resistance
- 2D motion
- Obstacles lie on the ground (y = 0)

---

## **2. Velocity Components**


$$

v\_{x0} = v_0 \cos\theta

$$


$$

v\_{y0} = v_0 \sin\theta

$$

---

## **3. Flight Phase (Projectile Motion)**

### **Position equations**

Horizontal:
$$

x(t) = v\_{x0} \, t

$$

Vertical:
$$

y(t) = y*0 + v*{y0} t - \frac{1}{2} g t^2

$$

### **Landing time**

Solve \(y(t) = 0\):


$$

y*0 + v*{y0} t - \frac{1}{2} g t^2 = 0

$$

Positive root:
$$

t*{\text{land}} = \frac{v*{y0} + \sqrt{v\_{y0}^2 + 2 g y_0}}{g}

$$

### **Horizontal landing position**


$$

x*{\text{land}} = v*{x0} \, t\_{\text{land}}

$$

---

## **4. Ground Sliding Phase**

After landing, the penguin slides with friction that removes:


$$

5 \text{ m/s per meter}

$$

### **Landing horizontal speed**


$$

v*{\text{land}} = v*{x0}

$$

### **Sliding distance**


$$

d*{\text{slide}} = \frac{v*{\text{land}}}{5}

$$

### **Total horizontal distance**


$$

D = x*{\text{land}} + d*{\text{slide}}

$$

---

## **5. Mines and Spikes**

### **Spike**
If the penguin reaches a spike at position \(x_s\):


$$

\text{score} = -100

$$

### **Mine**
If the penguin reaches a mine at position \(x_m\):

- Instant displacement:
$$

x \leftarrow x + 2

$$

- Instant velocity boost:
$$

v_x \leftarrow v_x + 4

$$

Then continue simulation with updated velocity.

---

## **6. Final Score**

If no spike is hit:


$$

\text{score} = \text{round}(100 \cdot D)

$$

(in centimeters)

---


$$


In [15]:
from math import cos, inf, radians, sin, sqrt

from utils.test_utilities import test

tests = [
    {
        "name": "test 1",
        "obstacles": [],
        "expected": (6829, 44),
    },
    {
        "name": "test 2",
        "obstacles": [(64.7, "spike")],
        "expected": (6466, 34),
    },
    {
        "name": "test 3",
        "obstacles": [(63.9, "mine"), (64.7, "spike")],
        "expected": (7022, 39),
    },
]


@test(tests=tests)
def solotion(obstacles: list[tuple[int, str]]) -> tuple[float, float]:
    obstacles.sort()

    if any(x_o == 0 for x_o, kind in obstacles if kind == "spike"):
        return -100, -100

    v_0, y_0 = 25, 1
    g = 9.81
    a_friction = 5

    d_max, theta_max = -100, -100

    for theta in range(-90, 90 + 1):
        v_x0 = v_0 * cos(radians(theta))
        v_y0 = v_0 * sin(radians(theta))

        t_land = (v_y0 + sqrt(v_y0**2 + 2 * g * y_0)) / g
        x_land = v_x0 * t_land

        v_land = v_x0
        d_slide = v_land / a_friction

        d = x_land + d_slide

        hit_spike = False
        for x_o, kind in obstacles:
            if x_land <= x_o <= d:
                if kind == "spike":
                    hit_spike = True
                    continue
                else:
                    d += 2 + 4 / a_friction
                    x_land = x_o + 2

        if hit_spike:
            continue

        if d > d_max:
            d_max = d
            theta_max = theta

    return round(d_max * 100), theta_max


Test test 1 passed, for solotion.
Test test 2 passed, for solotion.
Test test 3 passed, for solotion.
Success
